In [1]:
import os
from pathlib import Path
import pickle
from fastfusion import Specification
from fastfusion.mapper.FFM.exploration.mapper_multi_einsum import get_sims
from fastfusion.mapper.FFM.joining.simexplore import join_sims
from fastfusion.mapper.FFM.exploration.mapping_filter_tags import get_one_split_tag, get_ffmt_tag


ARCH_DIR          = Path('architecture/')
WORKLOAD_DIR      = Path('workloads/')
MAPPINGS_SIMS_DIR = Path('results/sims/')
MAPPINGS_DATA_DIR = Path('results/data/')


def one_split_tagger(compatibility):
    return get_one_split_tag(compatibility, "MainMemory")
NAME_TO_TAGGER = {
    'one_split': one_split_tagger
}


def get_experiment_name(tagger_name, arch_name: list[str], workload_name):
    combined_arch_name = '+'.join(arch_name)
    return f'matmul8_mixed.{tagger_name}.{combined_arch_name}.{workload_name}'


def get_sims_with_cache(tagger_name=None,
                        refresh_cache=False,
                        arch: list[str]=['snowcat'],
                        workload='matmuls8_mixed'):
    data_name  = get_experiment_name(tagger_name, arch, workload)
    sims_names = [get_experiment_name(tagger_name, [a], workload) for a in arch]

    result_pickle_name = MAPPINGS_DATA_DIR / f'{data_name}.pkl'
    if result_pickle_name.is_file() and not refresh_cache:
        with open(result_pickle_name, 'rb') as f:
            mappings = pickle.load(f)
            print(f'Loaded final results from cache {result_pickle_name}')
            return mappings

    if tagger_name is None:
        tagger = None
    else:
        tagger = NAME_TO_TAGGER[tagger_name]

    all_sims = []
    for a in arch:
        sims_name = get_experiment_name(tagger_name, [a], workload)
        sims_pickle_name = MAPPINGS_SIMS_DIR / f'{sims_name}.pkl'
        if sims_pickle_name.is_file() and not refresh_cache:
            with open(sims_pickle_name, 'rb') as f:
                sims = pickle.load(f)
                print(f'Loaded SIMs from {sims_pickle_name}')
                all_sims.append(sims)
                continue
        spec = Specification.from_yaml(ARCH_DIR / f'{a}.arch.yaml', WORKLOAD_DIR / f'{workload}.workload.yaml')
        spec.estimate_energy_area()
        workload = spec.workload
        renames = spec.renames
        flattened_architecture = spec.get_flattened_architecture()
        sims, decompress_data = get_sims(spec, flattened_architecture, tagger=tagger)

        with open(sims_pickle_name, 'wb') as f:
            pickle.dump(sims, f)

        all_sims.append(sims)

    combined_sims = {}
    for sims in all_sims:
        for einsum, sims_for_einsum in sims.items():
            if einsum not in combined_sims:
                combined_sims[einsum] = []
            combined_sims[einsum].extend(sims_for_einsum)

    mappings = join_sims(combined_sims, spec, flattened_architecture, drop_valid_reservations=False)
    # decompress_sims(mappings, decompress_data, workload.einsum_names)

    with open(result_pickle_name, 'wb') as f:
        pickle.dump(mappings, f)
        print(f'Saved results to cache {result_pickle_name}')

    return mappings

mappings_one_split = get_sims_with_cache('one_split')
mappings = get_sims_with_cache()
mappings_tileflow = get_sims_with_cache(tagger_name='one_split', arch=['snowcat_even'])
mappings_ffmt = get_sims_with_cache(arch=['snowcat_weight_tiled', 'snowcat_weight_untiled'])

# pr.disable()
# s = io.StringIO()
# ps = pstats.Stats(pr, stream=s).sort_stats('cumulative')
# ps.print_stats(30)  # Print top 30 time-consuming functions
# print(s.getvalue())

# TODO: Check for ranks not in the mapping and put them at the bottom
# TODO: What if there are no loops? 
# TODO: Set _must_exist for all backing storage nodes
# TODO: Constraint attacher
# TODO: Can't have tile size constraints on backing memory
# TODO: Einsum orders
# TODO: Copy Einsums
# TODO: Test dataflow constraints and order of storage nodes
# I'm doing the tile shape exploration now and I'm trying to understand this note. I think I understand what you're saying.
# Can I ask one thing from the constraint code? If the constraint is an equality, then just set the tile_shape attribute of the node (or factor or whatever is needed) to the value.
# The tile shape exploration assumes a particular mapspace (in most cases, tile shapes are factors of the full rank shape), so an equality may never be satisfied. E.g., if the constraint sets the tile shape equal to a non-factor value because you want a particular imperfect factorization, but that's never in the mapspace, then you'll get nothing.
# It's also a bit more efficient to just set the value and the explorer doesn't have to figure out the equality by trial-and-error. For other more complicated constraints, trial-and-error is better.

INFO        Loading yaml file architecture/snowcat.arch.yaml
INFO        Found top key variables in architecture/snowcat.arch.yaml
INFO        Found top key architecture in architecture/snowcat.arch.yaml
INFO        Loading yaml file workloads/matmuls8_mixed.workload.yaml
INFO        Found top key workload in workloads/matmuls8_mixed.workload.yaml
WARNING     Loading configuration file from /home/gilbertm/work/infrastructure/venv/fastfusion/config.yaml
INFO        Calculated "0.5" = 0.5.
Generating storage and loop choices for Einsum Matmul1: 20it [00:00, 331.56it/s]
Generating storage and loop choices for Einsum Matmul2: 36it [00:00, 431.85it/s]
Generating storage and loop choices for Einsum Matmul3: 36it [00:00, 439.86it/s]
Generating storage and loop choices for Einsum Matmul4: 36it [00:00, 467.14it/s]
Generating storage and loop choices for Einsum Matmul5: 36it [00:00, 440.15it/s]
Generating storage and loop choices for Einsum Matmul6: 36it [00:00, 423.01it/s]
Generating storage an

SIM Matmul1 tensors: {'T1'}
SIM Matmul2 tensors: {'T2', 'T1'}
SIM Matmul3 tensors: {'T3', 'T2'}
SIM Matmul4 tensors: {'T3', 'T4'}
SIM Matmul5 tensors: {'T4', 'T5'}
SIM Matmul6 tensors: {'T6', 'T5'}
SIM Matmul7 tensors: {'T7', 'T6'}
SIM Matmul8 tensors: {'T7'}


Grouping Partial Mappings: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 114/114 [00:00<00:00, 614.18it/s]


Initial consolidate and group: 15.80 seconds

Einsum Matmul2 (2/8)
Consolidating: 0.00 seconds


Grouping Partial Mappings: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [00:00<00:00, 470.96it/s]


...

Mapping merging: 0.03 seconds
Scaled runtime by 1.0. Runtime: 0.11
	Combining 7(8) x 112(64) -> 8
	Number of buckets for Einsum Matmul8: 8
	Number of mappings for Einsum Matmul8: 8
	Mappings per group for Einsum Matmul8: 1.0


Grouping Partial Mappings: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 1032.57it/s]



======== Total time ========
Initial consolidate and group: 0.34 seconds
Consolidating: 0.00 seconds
Combining: 0.00 seconds
Grouping: 0.00 seconds
Bucket merging: 0.01 seconds
Removing mappings that can't be combined later: 0.00 seconds
Mapping merging: 0.21 seconds

Total: 0.56 seconds

Saved results to cache results/data/matmul8_mixed.one_split.snowcat_even.matmuls8_mixed.pkl


INFO        Loading yaml file architecture/snowcat_weight_tiled.arch.yaml
INFO        Found top key variables in architecture/snowcat_weight_tiled.arch.yaml
INFO        Found top key architecture in architecture/snowcat_weight_tiled.arch.yaml
INFO        Loading yaml file workloads/matmuls8_mixed.workload.yaml
INFO        Found top key workload in workloads/matmuls8_mixed.workload.yaml
WARNING     Loading configuration file from /home/gilbertm/work/infrastructure/venv/fastfusion/config.yaml
Generating storage and loop choices for Einsum Matmul1: 0it [00:00, ?it/s]


ParseError: ParseError in : name 'weight' is not defined. Set expression: "~weight". Symbol table:
	Nothing: InvertibleSet(frozenset())
	All: InvertibleSet(frozenset({'T0', 'W0', 'T1'}))
	Inputs: InvertibleSet(frozenset({'T0', 'W0'}))
	Outputs: InvertibleSet(frozenset({'T1'}))
	Intermediates: InvertibleSet(frozenset({'T1'}))
	Shared: InvertibleSet(frozenset({'T1'}))
	T0: InvertibleSet(frozenset({'T0'}))
	W0: InvertibleSet(frozenset({'W0'}))
	T1: InvertibleSet(frozenset({'T1'}))
	m: InvertibleSet(frozenset({'m'}))
	n0: InvertibleSet(frozenset({'n0'}))
	n1: InvertibleSet(frozenset({'n1'}))
	MainMemory: InvertibleSet(frozenset({'T0', 'W0'}))
	GlobalBuffer: InvertibleSet(frozenset({'T0', 'W0', 'T1'}))

In [12]:
import copy
import re
from fastfusion.frontend import Workload
from fastfusion.frontend.mapping import Iteration, Mapping, Nested, Split, Storage
from fastfusion.visualization.interactive import plotly_show
from fastfusion.mapper.FFM.visualization import make_mapping

einsum_names = Workload.from_yaml(WORKLOAD_DIR / 'matmuls8_mixed.workload.yaml').einsum_names

plotly_show(mappings_tileflow.data, "RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy", logscales=True, einsum_names=einsum_names)

INFO        Loading yaml file workloads/matmuls8_mixed.workload.yaml
INFO        Found top key workload in workloads/matmuls8_mixed.workload.yaml
WARNING     Trying to parse a single element dictionary as a Workload. 


    'data': [{'line': {'shape': 'hv'},
              'marker': {'symbol': 'circl…

In [3]:
for i in range(1,5):
    print(i)
    for node in mappings_one_split.data.iloc[-i]['Matmul1___MAPPING'].nodes:
        print(node)

1
W0 in MainMemory
type=None tensor='W0' memory='MainMemory'
type=None tensor='W0' memory='MainMemory'
T0 in MainMemory
type=None tensor='T0' memory='MainMemory'
type=None tensor='T0' memory='MainMemory'
for n1 shape tileshape0
W0 in GlobalBuffer
type=None tensor='W0' memory='GlobalBuffer'
type=None tensor='W0' memory='GlobalBuffer'
for m shape tileshape1
T1 in GlobalBuffer
type=None tensor='T1' memory='GlobalBuffer'
type=None tensor='T1' memory='GlobalBuffer'
for m shape 1
for n0 shape 1
T0 in GlobalBuffer
type=None tensor='T0' memory='GlobalBuffer'
type=None tensor='T0' memory='GlobalBuffer'
for n1 shape 1
Einsum Matmul1
2
W0 in MainMemory
type=None tensor='W0' memory='MainMemory'
type=None tensor='W0' memory='MainMemory'
T0 in MainMemory
type=None tensor='T0' memory='MainMemory'
type=None tensor='T0' memory='MainMemory'
for n1 shape tileshape0
W0 in GlobalBuffer
type=None tensor='W0' memory='GlobalBuffer'
type=None tensor='W0' memory='GlobalBuffer'
for m shape tileshape1
T1 in Globa

In [5]:
plotly_show(mappings_one_split.data, "RESOURCE_GlobalBuffer_LEVEL_0", "metric_Energy", logscales=True, einsum_names=spec.workload.einsum_names)

    'data': [{'line': {'shape': 'hv'},
              'marker': {'symbol': 'circl…